# Выбор оптимальной конфигурации модели

In [1]:
# standard libraries
import pandas as pd
import sys
import os
import yaml
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format','{:_.2f}'.format)
if 'drive' not in os.listdir():
    from google.colab import drive
    drive.mount('/content/drive')

path_to_folder = '/content/drive/MyDrive/psb_hack'

Mounted at /content/drive


# Выбор лучшей модели по метрикам по продукту

In [32]:
main_metrics = ['precision', 'recall', 'f1', 'roc-auc original', 'roc-auc_hack']
test_set_name = 'val_oot'

def read_metric_table(file_name):
    table = pd.read_excel(file_name)
    table['config_model'] = file_name.split('/')[-1].split('.')[0].replace('_metrics','')
    return table

# Загрзука метрик

In [33]:
# файлы с метриками по модели суммы кредита
metrics = os.listdir(os.path.join(path_to_folder, 'metrics'))
# соберем их в единый файл с указанием модели
all_metrics = pd.concat(list(map(lambda x: read_metric_table(os.path.join(path_to_folder,'metrics', x)),metrics))).reset_index(drop=True)

In [41]:
all_metrics.sample(5)

,dataset,target,group,measurement,N,accuracy,precision,recall,f1,roc-auc original,roc-auc_hack,pr-auc,target_label_count_0,target_label_count_1,pred_label_count_0,pred_label_count_1,auc,config_model
8,train,target,has_prepayment,0,3111,0.91,0.93,0.92,0.92,0.97,0.90,0.98,1172,1939,1181,1_930.00,0.97,psb_hack_lama_v541
21,oof_val,target,region,2,7219,0.89,0.89,0.51,0.65,0.84,0.75,0.73,5770,1449,6379,840.00,0.84,psb_hack_lama_v541
11,oot_val,target,region,1,1969,0.92,0.92,0.58,0.71,0.85,0.78,0.74,1616,353,1746,223.00,0.85,psb_hack_lama_v541
28,oof_val,target,has_prepayment,1,5692,0.90,0.85,0.02,0.04,0.68,0.51,0.21,5103,589,5679,13.00,0.68,psb_hack_lama_v541
12,oot_val,target,region,2,999,0.89,0.97,0.36,0.52,0.78,0.68,0.62,834,165,938,61.00,0.78,psb_hack_lama_v541


### Выбор лучшей конфигурации на test

In [35]:
oot = all_metrics.loc[all_metrics['dataset']=='oot_val'] \
              .groupby(['group','measurement'])[main_metrics].idxmin().applymap(lambda x: all_metrics.iloc[int(x),-1])

In [36]:
oot

precision  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                       recall  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                           f1  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                             roc-auc original  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         1                           psb_hack_lama_v541   
               2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                 roc-auc_hack  
group          measurement                                     
has_prepayment 0                           psb_hack_lama_v541  
               1                           psb_hack_lama_v541  
region         1                           psb_hack_lama_v541  
               2                           psb_hack_lama_v541  
Источник       ostrovok.ru                 psb_hack_lama_v541  
               Бронирование из экстранета  psb_hack_lama_v541  
               Официальный сайт            psb_hack_lama_v541  
               Программа лояльности        psb_hack_lama_v541  
               Яндекс.Путешествия          psb_hack_lama_v541

### Выбор лучшей конфигурации на val

In [38]:
oof = all_metrics.loc[all_metrics['dataset']=='oof_val'].groupby(['group','measurement'])[main_metrics].idxmin().applymap(lambda x: all_metrics.iloc[int(x),-1])

In [39]:
oof

precision  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                       recall  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                           f1  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                             roc-auc original  \
group          measurement                                      
has_prepayment 0                           psb_hack_lama_v541   
               1                           psb_hack_lama_v541   
region         2                           psb_hack_lama_v541   
Источник       ostrovok.ru                 psb_hack_lama_v541   
               Бронирование из экстранета  psb_hack_lama_v541   
               Официальный сайт            psb_hack_lama_v541   
               Программа лояльности        psb_hack_lama_v541   
               Яндекс.Путешествия          psb_hack_lama_v541   

                                                 roc-auc_hack  
group          measurement                                     
has_prepayment 0                           psb_hack_lama_v541  
               1                           psb_hack_lama_v541  
region         2                           psb_hack_lama_v541  
Источник       ostrovok.ru                 psb_hack_lama_v541  
               Бронирование из экстранета  psb_hack_lama_v541  
               Официальный сайт            psb_hack_lama_v541  
               Программа лояльности        psb_hack_lama_v541  
               Яндекс.Путешествия          psb_hack_lama_v541